In [ ]:
%run -p train_cnf.py --data mnist --dims 64,64,64 --strides 1,1,1,1 --num_blocks 2 --layer_type concat --multiscale True --rademacher True --batch_size 8000 --test_batch_size 8000 --save experiments/cnf_nocond_bs8K --conditional False

/tancode/repos/tan-ffjord/train_cnf.py
import argparse
import os
import time
import numpy as np

import torch
import torch.optim as optim
import torchvision.datasets as dset
import torchvision.transforms as tforms
from torchvision.utils import save_image

import lib.layers as layers
import lib.utils as utils
import lib.odenvp as odenvp
import lib.multiscale_parallel as multiscale_parallel

from train_misc import standard_normal_logprob
from train_misc import set_cnf_options, count_nfe, count_parameters, count_total_time
from train_misc import add_spectral_norm, spectral_norm_power_iteration
from train_misc import create_regularization_fns, get_regularization, append_regularization_to_log

# go fast boi!!
torch.backends.cudnn.benchmark = True
SOLVERS = ["dopri5", "bdf", "rk4", "midpoint", 'adams', 'explicit_adams']
parser = argparse.ArgumentParser("Continuous Normalizing Flow")
parser.add_argument("--data", choices=["mnist", "svhn", "cifar10", 'lsun_church'], type=str, default="mnist")


do _generate
do _generate
do _generate
do _generate
do _generate
do _generate
do _generate
do _generate


===> Using batch size 8000. Total 7 iterations/epoch.
Iter 0007 | Time 28.4405(58.0695) | Bit/dim 24.5518(25.7252) | Steps 410(410.00) | Grad Norm 195.2933(205.4546) | Total Time 10.00(10.00)
Iter 0008 | Time 26.5125(57.1228) | Bit/dim 24.2112(25.6798) | Steps 410(410.00) | Grad Norm 192.6661(205.0710) | Total Time 10.00(10.00)
Iter 0009 | Time 26.6585(56.2089) | Bit/dim 23.8490(25.6249) | Steps 410(410.00) | Grad Norm 189.7350(204.6109) | Total Time 10.00(10.00)
Iter 0010 | Time 26.6223(55.3213) | Bit/dim 23.4132(25.5585) | Steps 410(410.00) | Grad Norm 185.8377(204.0477) | Total Time 10.00(10.00)
Iter 0011 | Time 26.2271(54.4484) | Bit/dim 22.9708(25.4809) | Steps 410(410.00) | Grad Norm 181.3509(203.3668) | Total Time 10.00(10.00)
Iter 0012 | Time 26.4226(53.6077) | Bit/dim 22.4924(25.3912) | Steps 410(410.00) | Grad Norm 177.9913(202.6055) | Total Time 10.00(10.00)
Iter 0013 | Time 26.8054(52.8036) | Bit/dim 22.0062(25.2897) | Steps 410(410.00) | Grad Norm 172.7741(201.7106) | Tota

do _generate
do _generate
do _generate
do _generate
do _generate
do _generate
do _generate
do _generate


===> Using batch size 8000. Total 7 iterations/epoch.
Iter 0014 | Time 27.3366(52.0396) | Bit/dim 21.4764(25.1753) | Steps 410(410.00) | Grad Norm 168.6384(200.7184) | Total Time 10.00(10.00)
Iter 0015 | Time 26.4782(51.2727) | Bit/dim 20.9535(25.0486) | Steps 410(410.00) | Grad Norm 162.9912(199.5866) | Total Time 10.00(10.00)
Iter 0016 | Time 26.5420(50.5308) | Bit/dim 20.4183(24.9097) | Steps 410(410.00) | Grad Norm 157.7711(198.3321) | Total Time 10.00(10.00)
Iter 0017 | Time 26.6628(49.8148) | Bit/dim 19.8417(24.7577) | Steps 410(410.00) | Grad Norm 152.6234(196.9609) | Total Time 10.00(10.00)
Iter 0018 | Time 27.3651(49.1413) | Bit/dim 19.2992(24.5939) | Steps 410(410.00) | Grad Norm 146.3866(195.4436) | Total Time 10.00(10.00)
Iter 0019 | Time 26.9746(48.4763) | Bit/dim 18.7327(24.4181) | Steps 410(410.00) | Grad Norm 140.5430(193.7966) | Total Time 10.00(10.00)
Iter 0020 | Time 26.7336(47.8240) | Bit/dim 18.1796(24.2309) | Steps 410(410.00) | Grad Norm 134.5533(192.0193) | Tota

...

===> Using batch size 8000. Total 7 iterations/epoch.
Iter 1617 | Time 35.1319(36.0554) | Bit/dim 1.2395(1.2558) | Steps 494(496.69) | Grad Norm 2.0060(11.6311) | Total Time 10.00(10.00)
Iter 1618 | Time 35.3126(36.0331) | Bit/dim 1.2404(1.2554) | Steps 488(496.43) | Grad Norm 6.6890(11.4828) | Total Time 10.00(10.00)
Iter 1619 | Time 36.5393(36.0483) | Bit/dim 1.2421(1.2550) | Steps 500(496.54) | Grad Norm 11.6777(11.4887) | Total Time 10.00(10.00)
Iter 1620 | Time 34.4031(35.9989) | Bit/dim 1.2540(1.2549) | Steps 488(496.28) | Grad Norm 15.0136(11.5944) | Total Time 10.00(10.00)
Iter 1621 | Time 36.5161(36.0145) | Bit/dim 1.2554(1.2550) | Steps 506(496.57) | Grad Norm 16.6252(11.7453) | Total Time 10.00(10.00)
Iter 1622 | Time 36.6642(36.0340) | Bit/dim 1.2632(1.2552) | Steps 488(496.32) | Grad Norm 16.8945(11.8998) | Total Time 10.00(10.00)
Iter 1623 | Time 36.4466(36.0463) | Bit/dim 1.2520(1.2551) | Steps 506(496.61) | Grad Norm 13.9638(11.9617) | Total Time 10.00(10.00)
validating

do _generatedo _generate

do _generate
do _generate
do _generate
do _generate
do _generate
do _generate


===> Using batch size 8000. Total 7 iterations/epoch.
Iter 1624 | Time 34.8864(36.0115) | Bit/dim 1.2402(1.2547) | Steps 488(496.35) | Grad Norm 9.1421(11.8772) | Total Time 10.00(10.00)
Iter 1625 | Time 36.5524(36.0278) | Bit/dim 1.2413(1.2543) | Steps 500(496.46) | Grad Norm 3.9714(11.6400) | Total Time 10.00(10.00)
Iter 1626 | Time 36.5198(36.0425) | Bit/dim 1.2374(1.2538) | Steps 494(496.38) | Grad Norm 2.9220(11.3784) | Total Time 10.00(10.00)
Iter 1627 | Time 36.8854(36.0678) | Bit/dim 1.2423(1.2534) | Steps 488(496.13) | Grad Norm 8.3359(11.2872) | Total Time 10.00(10.00)
Iter 1628 | Time 38.3540(36.1364) | Bit/dim 1.2501(1.2533) | Steps 506(496.43) | Grad Norm 13.0872(11.3412) | Total Time 10.00(10.00)
Iter 1629 | Time 36.1221(36.1360) | Bit/dim 1.2587(1.2535) | Steps 488(496.18) | Grad Norm 17.0592(11.5127) | Total Time 10.00(10.00)
Iter 1630 | Time 37.2258(36.1687) | Bit/dim 1.2558(1.2535) | Steps 506(496.47) | Grad Norm 19.4349(11.7504) | Total Time 10.00(10.00)
validating..

do _generate
do _generate
do _generate
do _generate
do _generate
do _generate
do _generate
do _generate


===> Using batch size 8000. Total 7 iterations/epoch.
Iter 1631 | Time 34.7370(36.1257) | Bit/dim 1.2697(1.2540) | Steps 488(496.22) | Grad Norm 19.7917(11.9916) | Total Time 10.00(10.00)
